In [7]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.preprocessing import LabelEncoder, StandardScaler

raw_path = r"C:\Users\shafn\OneDrive\Desktop\CUSTOMER CHURN PREDICTION AND ANALYSIS SYSTEM\WA_Fn-UseC_-Telco-Customer-Churn(1).csv"
df = pd.read_csv(raw_path)

display(df.head())
df.info()




,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:
print("Missing per column:\n", df.isnull().sum())
print("Duplicates:", df.duplicated().sum())

Missing per column:
 customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64
Duplicates: 0


In [9]:
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

print(df['TotalCharges'].isnull().sum())

11


In [10]:
df['TotalCharges'] = df.apply(
    lambda row: row['MonthlyCharges'] * row['tenure'] if pd.isnull(row['TotalCharges']) else row['TotalCharges'],
    axis=1
)


In [11]:
df = df.drop_duplicates()

for col in df.columns:
    if df[col].dtype.kind in 'biufc': # numeric
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])


In [12]:

num_cols = [c for c in df.columns if df[c].dtype.kind in 'biufc' and c != 'Churn']
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    df = df[(df[col] >= lower) & (df[col] <= upper)]


In [13]:

if 'Churn' in df.columns:
    df['Churn'] = df['Churn'].map({'Yes':1, 'No':0}).astype(int)


le = LabelEncoder()
cat_cols = [c for c in df.columns if df[c].dtype == 'object' and c != 'Churn']

for c in cat_cols:
    if df[c].nunique() == 2:
        df[c] = le.fit_transform(df[c])
        
multi_cat = [c for c in cat_cols if df[c].nunique() > 2]
df = pd.get_dummies(df, columns=multi_cat, drop_first=True)


In [14]:
scaler = StandardScaler()
num_cols = [c for c in df.columns if df[c].dtype.kind in 'biufc' and c != 'Churn']
df[num_cols] = scaler.fit_transform(df[num_cols])


In [6]:
import os
import pandas as pd

raw_path = r"C:\Users\shafn\OneDrive\Desktop\CUSTOMER CHURN PREDICTION AND ANALYSIS SYSTEM\WA_Fn-UseC_-Telco-Customer-Churn(1).csv"
df = pd.read_csv(raw_path)

os.makedirs("data/processed", exist_ok=True)

clean_csv = "data/processed/clean_churn.csv"
df.to_csv(clean_csv, index=False)
print("Saved cleaned csv:", clean_csv)

import sqlite3
conn = sqlite3.connect("churn.db")
df.to_sql("customers", conn, if_exists="replace", index=False)
conn.close()
print("Saved SQLite DB: churn.db (table customers)")



Saved cleaned csv: data/processed/clean_churn.csv
Saved SQLite DB: churn.db (table customers)
